# Select Data and Reconstruct Trajectory

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
import functionsMasterProjectMeinhart as fmpm

%matplotlib auto

Using matplotlib backend: TkAgg


## Select file, time range and sampling rate

In [2]:
# select file (csv)
file_dir  = r'E:\Physio_Data\Subject_01'
file_name = 'subject01.csv'

# selecet time range [min:sec]
start_min_sec = '02:20.0' # Raises Oblique
stop_min_sec  = '02:37.1'

sampling_rate = 256 # [Hz]

In [61]:
# select file (csv)
file_dir  = r'E:\Physio_Data\Subject_01'
file_name = 'subject01.csv'

# selecet time range [min:sec]
start_min_sec = '21:24.0' # Raises Side
stop_min_sec  = '21:40.2'

sampling_rate = 256 # [Hz]

In [4]:
# select file (csv)
file_dir  = r'E:\Physio_Data\Subject_01'
file_name = 'subject01.csv'

# selecet time range [min:sec]
start_min_sec = '18:58.0' # Biceps Curls
stop_min_sec  = '19:12.0'

sampling_rate = 256 # [Hz]

In [5]:
# select file (csv)
file_dir  = r'E:\Physio_Data\Subject_01'
file_name = 'subject01.csv'

# selecet time range [min:sec]
start_min_sec = '25:48.7' # PNF Diagonal 1
stop_min_sec  = '26:04.6'

sampling_rate = 256 # [Hz]

In [6]:
# select file (csv)
file_dir  = r'E:\Physio_Data\Subject_05'
file_name = 'subject05.csv'

# selecet time range [min:sec]
start_min_sec = '33:49.3' # PNF Diagonal 1
stop_min_sec  = '34:01.5'

sampling_rate = 256 # [Hz]

In [2]:
# select file (csv)
file_dir  = r'E:\Physio_Data\Subject_06'
file_name = 'subject06.csv'

# selecet time range [min:sec]
start_min_sec = '28:47' # PNF Diagonal 1
stop_min_sec  = '29:00'

sampling_rate = 256 # [Hz]

## Plot raw and filtered data of selected time range $\rightarrow$ calculate trajectory of new selected time range

In [3]:
# split time string and convert to float
start_min = float(start_min_sec.split(':')[0])
start_sec = float(start_min_sec.split(':')[1])
stop_min = float(stop_min_sec.split(':')[0])
stop_sec = float(stop_min_sec.split(':')[1])

# start and stop time in seconds
start_time = start_min*60 + start_sec # [s]
stop_time = stop_min*60 + stop_sec # [s]

# join data path
data_path = os.path.join(file_dir, file_name)

# get data from selected file
sensor_data = fmpm.get_sensor_data(in_file=data_path,
                                   sampling_rate=sampling_rate,
                                   start_time=start_time,
                                   stop_time=stop_time)

# filter acceleration and angualar velocity with Savitzky-Golay filter and plot data
window_length = 51
polyorder = 3
acc_filt = savgol_filter(sensor_data['Acc'],window_length, polyorder, axis=0)
gyr_filt = savgol_filter(sensor_data['Gyr'],window_length, polyorder, axis=0)


# plot raw data (acceleration and angular velocity) !!! only works with %matplotlib auto
fig, axs = plt.subplots(2,2,figsize=(16,8))
plt.sca(axs[0,0])
fmpm.plot_signal(sensor_data['Acc'],sensor_data['time'],
            Title='Acceleration Profile',
            g_to_ms2=True)
plt.gca().set_xlabel('')
plt.sca(axs[0,1])
fmpm.plot_signal(sensor_data['Gyr'],sensor_data['time'],
            Title='Angular Velocity Profile',
            xLabel=r'$time \enspace [s]$',
            yLabel=r'$vel \enspace [\frac{deg}{s}]$')
plt.gca().set_xlabel('')

# plot filtered data (acceleration and angular velocity)
plt.sca(axs[1,0])
fmpm.plot_signal(acc_filt,sensor_data['time'],
            Title=r'Acceleration Profile $\rightarrow$ filtered',
            g_to_ms2=True)
plt.sca(axs[1,1])
fmpm.plot_signal(gyr_filt,sensor_data['time'],
            Title=r'Angular Velocity Profile $\rightarrow$ filtered',
            xLabel=r'$time \enspace [s]$',
            yLabel=r'$vel \enspace [\frac{deg}{s}]$')
fig.suptitle('Select time range via mouse clicks ...', fontsize=20)
plt.show()


# select two points for time range
selected_points = plt.ginput(2)

# check if two points are selected
if np.shape(selected_points)[0] is 2:
    
    # selected start and stop time
    start_time_sel = selected_points[0][0]
    stop_time_sel  = selected_points[1][0]
    
    # change start and stop time if start time is higher than stop time
    if start_time_sel > stop_time_sel:
        puffer = start_time_sel
        start_time_sel = stop_time_sel
        stop_time_sel = puffer
    
    # calculate corresponding index   !!! minus general start time
    start_time_sel_index = int((start_time_sel-start_time)*sampling_rate)
    stop_time_sel_index  = int((stop_time_sel-start_time)*sampling_rate)
    
    # refresh title of figure and show selected time range
    fig.suptitle(r'Selected time range: {0:02d}:{1:05.2f} $\rightarrow$ {2:02d}:{3:05.2f}'.format(
        int(start_time_sel/60),start_time_sel%60,int(stop_time_sel/60),stop_time_sel%60),fontsize=20)
    
    # mark selected area
    for ii in range(2):
        for jj in range(2):
            plt.sca(axs[ii,jj])
            plt.axvspan(start_time_sel, stop_time_sel, color='y', alpha=0.3, lw=0)
    
    # get selected filtered acceleration and angular velocity
    acc_filt_selected = acc_filt[start_time_sel_index:stop_time_sel_index,:]
    gyr_filt_selected = gyr_filt[start_time_sel_index:stop_time_sel_index,:]
    # time_selected = sensor_data['time'][start_time_sel_index:stop_time_sel_index] not necessary
    
    # calculate trajectory in an upright frame
    traj_data_filt = fmpm.calc_trajectory(acc_filt_selected,
                                          gyr_filt_selected,
                                          sampling_rate=sampling_rate)
    
    # plot trajectory in an upright frame as well as the initial orientation
    fmpm.plot_trajectory(pos_data=traj_data_filt['pos'],
                         scale_plot_section = 0.7,
                         rotmat_upright_init=traj_data_filt['R_upright_init'],
                         scale_arrow=0.2,
                         fig_size=(8,8))

E:\Programme\WinPython-64bit-3.6.2.0Qt5\python-3.6.2.amd64\lib\site-packages\matplotlib\backend_bases.py:2453: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)
